### create array

In [12]:
import numpy as np
import pandas as pd

# Choose frequency, binsize, longitude, latitude
freq = 'D'
binsize = 5
longitude_W = 134  # minimum is 134
longitude_E = 174  # maximum is 174
latitude_S = 10  # minimum is 10
latitude_N = 60  # minimum is 60

# load earthquake data for defined area
data = pd.read_csv('data/Japan_10_60_134_174_1973_2023_V2.csv', index_col=0)
data['Time'] = pd.to_datetime(data.Time)
data = data[(data.Longitude >= longitude_W) & (data.Longitude <= longitude_E) & (data.Latitude >= latitude_S) & (data.Latitude <= latitude_N)]
data.set_index('Time', inplace=True)
df = data.sort_index()

# Bin the longitude and latitude values into 2x2 degree bins
df['Longitude_bin'] = pd.cut(df['Longitude'], bins=np.arange(longitude_W, longitude_E + 1, binsize))  # Change bin size to 2 degrees
df['Latitude_bin'] = pd.cut(df['Latitude'], bins=np.arange(latitude_S, latitude_N + 1, binsize))  # Change bin size to 2 degrees

# Group the data by longitude bin, latitude bin, depth bin, and day, and compute the maximum magnitude within each group
grouped = df.groupby(['Longitude_bin', 'Latitude_bin', pd.Grouper(freq=freq, level="Time")]).agg({'Magnitude': 'max', 'Depth': 'mean'})
grouped = grouped.unstack().fillna(0)

# Reshape the resulting data into a tensor with shape (time, longitude, latitude, 4)
time = len(grouped.columns.levels[1])
longitude = len(grouped.index.levels[0])
latitude = len(grouped.index.levels[1])
tensor = np.zeros((time, longitude, latitude, 4))

for t in range(time):
    tensor[t, :, :, 0] = grouped['Magnitude'].iloc[:, t].values.reshape(longitude, latitude)
    tensor[t, :, :, 3] = grouped['Depth'].iloc[:, t].values.reshape(longitude, latitude)

latitude_values = np.linspace(latitude_S, latitude_N - binsize, latitude) + binsize / 2
longitude_values = np.linspace(longitude_W, longitude_E - binsize, longitude) + binsize / 2

tensor[:, :, :, 1] = np.repeat(latitude_values, longitude).reshape(latitude, longitude).T
tensor[:, :, :, 2] = np.tile(longitude_values, latitude).reshape(latitude, longitude).T

# Rotate dimensions corresponding to 20 and 25, 90 degrees anti-clockwise
tensor = np.transpose(tensor, axes=(0, 2, 1, 3))
tensor = np.flip(tensor, axis=1)

# Print the shape of the tensor
print(tensor.shape)

# Reshape tensor into matrix
matrix = np.reshape(tensor[:,:,:,0], (tensor.shape[0], -1))

print(matrix.shape)


(18260, 10, 8, 4)
(18260, 80)


In [ ]:
select = matrix.flatten() >= 0
print((matrix.flatten()[select] >= 6).mean())
alpha = 1 - (matrix.flatten()[select] >= 6).mean()
print(alpha)

### split the data

In [13]:
from sklearn.model_selection import train_test_split

# split data in train, val, test

train, val_test = train_test_split(matrix, test_size=.3, shuffle=False, random_state=43)
val, test = train_test_split(val_test, test_size=.5, shuffle=False, random_state=43)

### create sliding window datasets

In [18]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import tensorflow as tf

def dataset_generator(data, seq_length, samp_rate, stride, cutoff):

  input_data = data
  target_data = data
  dataset = timeseries_dataset_from_array(input_data, targets=None, sequence_length=seq_length, sampling_rate=samp_rate, sequence_stride=stride, shuffle=False, batch_size=len(data))

  return dataset

# Set lookback timewindow
lookback_window = 30 * 6
cutoff = 4.5
future_timesteps = 30
timewindow = lookback_window + future_timesteps
samp_rate = 1
stride = future_timesteps

train_dataset = dataset_generator(train, timewindow, samp_rate, stride, cutoff)
val_dataset = dataset_generator(val, timewindow, samp_rate, stride, cutoff)
test_dataset = dataset_generator(test, timewindow, samp_rate, stride, cutoff)

# Create train set
for batch in train_dataset:
    X_train = batch[:,:-future_timesteps,:]
    # X_train = np.reshape(X_train, (X_train.shape[0], -1))

    y_train = batch[:,-future_timesteps:,:] >= cutoff
    y_train = np.max(y_train, axis=1).astype(int)
    # y_train = np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[1])).astype(int)

# y_train = tf.reshape(y_train, shape=[y_train.shape[0], 1, y_train.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_train = tf.cast(tf.reduce_max(y_train, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create validation set
for batch in val_dataset:
    X_val = batch[:,:-future_timesteps,:]
    # X_val = np.reshape(X_val, (X_val.shape[0], -1))

    y_val = batch[:,-future_timesteps:,:] >= cutoff
    y_val = np.max(y_val, axis=1).astype(int)
    # y_val = np.reshape(y_val, (y_val.shape[0], 1, y_val.shape[1])).astype(int)

# y_val = tf.reshape(y_val, shape=[y_val.shape[0], 1, y_val.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_val = tf.cast(tf.reduce_max(y_val, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create test set
for batch in test_dataset:
    X_test = batch[:,:-future_timesteps,:]
    # X_test = np.reshape(X_test, (X_test.shape[0], -1))

    y_test = batch[:,-future_timesteps:,:] >= cutoff
    y_test = np.max(y_test, axis=1).astype(int)
    # y_test = np.reshape(y_test, (y_test.shape[0], 1, y_test.shape[1])).astype(int)

# y_test = tf.reshape(y_test, shape=[y_test.shape[0], 1, y_test.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_test = tf.cast(tf.reduce_max(y_test, axis=2, keepdims=True) > 0, dtype=tf.int32)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(420, 180, 80) (420, 80)
(85, 180, 80) (85, 80)
(85, 180, 80) (85, 80)


### LSTM model build

In [ ]:
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Flatten, Input, TimeDistributed, Dropout, RepeatVector, BatchNormalization
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, ConvLSTM1D
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
import keras
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import backend as K

keras.backend.clear_session()

# define model
model = Sequential()

model.add(BatchNormalization())
model.add(LSTM(64, activation='tanh',
               return_sequences=True,
               input_shape=(X_train.shape[1:]),
               kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
               bias_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
               dropout=0.4,
               ))
               
model.add(BatchNormalization())

model.add(LSTM(64, activation='tanh', return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation="sigmoid"))

opt = keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryFocalCrossentropy( apply_class_balancing=True, alpha=.95, gamma=5)
model.compile(optimizer=opt, loss=loss, metrics=[keras.metrics.Precision(), keras.metrics.Recall()])
model.build(input_shape=X_train.shape)
model.summary()


### fit model


In [ ]:
# early stopping
callback = EarlyStopping(monitor='val_loss', patience=3)
# fit model
history = model.fit(x=X_train,
                    y=y_train,
                    validation_data=(X_val,y_val),
                    batch_size=128,
                    epochs=500,
                    verbose=1,
                    callbacks=[callback],
                    shuffle=False)

### run 10 iterations of the model (and save it to disk)

In [ ]:
for i in range(1,11):
    keras.backend.clear_session()
    # define model
    model = Sequential()

    model.add(BatchNormalization())
    model.add(LSTM(64, activation='tanh',
                return_sequences=True,
                input_shape=(X_train.shape[1:]),
                kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
                bias_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
                dropout=0.4,
                ))
                
    model.add(BatchNormalization())

    model.add(LSTM(64, activation='tanh', return_sequences=False))
    model.add(BatchNormalization())
    model.add(Dense(y_train.shape[1], activation="sigmoid"))

    opt = keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.BinaryFocalCrossentropy( apply_class_balancing=True, alpha=.95, gamma=5)
    model.compile(optimizer=opt, loss=loss, metrics=[keras.metrics.Precision(), keras.metrics.Recall()])
    model.build(input_shape=X_train.shape)

    # early stopping
    callback = EarlyStopping(monitor='val_loss', patience=3)


    history = model.fit(x=X_train,
                        y=y_train,
                        validation_data=(X_val,y_val),
                        batch_size=128,
                        epochs=500,
                        verbose=1,
                        callbacks=[callback],
                        shuffle=False)
    
    model.save('models/LSTM_5x5_72M_1M_M6_iteration_{}'.format(i))


In [ ]:
# model.save('models/LSTM_5x5_72M_1M_M4.5')

### import model

In [ ]:
import keras
model = keras.models.load_model('models/LSTM_5x5_72M_1M_M4.5_iteration_1')

### predict

In [ ]:
y_pred = model.predict(X_test)

### evaluation metrics

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
y_val = y_test
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(np.array(y_val).flatten(), y_pred.flatten() >= .5)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(np.array(y_val).flatten(), y_pred.flatten() >= .5)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(np.array(y_val).flatten(), y_pred.flatten() >= .5)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(np.array(y_val).flatten(), y_pred.flatten() >= .5)
print('F1 score: %f' % f1)

class_names = ['M<{}'.format(cutoff), 'M>={}'.format(cutoff)]

print(classification_report(np.array(y_val).flatten(), y_pred.flatten() >= .5, target_names=class_names))

### calculate mean(std) on the 10 iterations

In [19]:
import keras
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

P = []
R = []
F1 = []

for i in range(1,11):
    model = keras.models.load_model('models/LSTM_5x5_6M_1M_M4.5_iteration_{}'.format(i))

    y_pred = model.predict(X_test)

    # precision tp / (tp + fp)
    precision = precision_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
    P.append(precision)
    # recall: tp / (tp + fn)
    recall = recall_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
    R.append(recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
    F1.append(f1)

    if i == 10:
        df = pd.DataFrame((P,R,F1)).T.rename(columns={0: "P", 1: "R", 2: "F1"})
        df.to_csv("LSTM_4.5_6M_iterations.csv", index=False)
        print('Precision Mean: %f' % np.mean(P), 'Std: %f' % np.std(P))
        print('Recall Mean: %f' % np.mean(R), 'Std: %f' % np.std(R))
        print('F1 Mean: %f' % np.mean(F1), 'Std: %f' % np.std(F1))
        print(df)

3/3 [==============================] - 0s 25ms/step
Precision Mean: 0.758373 Std: 0.003853
Recall Mean: 0.869268 Std: 0.002592
F1 Mean: 0.810031 Std: 0.001370
          P         R        F1
0  0.751196  0.875261  0.808497
1  0.757282  0.869686  0.809601
2  0.761148  0.868293  0.811198
3  0.754534  0.869686  0.808028
4  0.761614  0.868293  0.811462
5  0.757264  0.871777  0.810496
6  0.762431  0.865505  0.810705
7  0.764128  0.866899  0.812276
8  0.754991  0.869686  0.808290
9  0.759146  0.867596  0.809756


### confusion matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.set(rc={'figure.figsize':(5,5)})
p = sns.heatmap(confusion_matrix(np.array(y_val).flatten(), y_pred.flatten() >= 0.5), annot=True, fmt='g')
p.set_xlabel("Predicted")
p.set_ylabel("True")
p.set_title("LSTM")
p.xaxis.set_ticklabels(['M<{}'.format(cutoff), 'M>={}'.format(cutoff)], ha="center", va="center")
p.yaxis.set_ticklabels(['M<{}'.format(cutoff), 'M>={}'.format(cutoff)], rotation=0, va="center")
plt.savefig("graphs/CM_LSTM_5x5_1M_M45")

### ROC curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# y_pred = scaler.inverse_transform(y_prob)
# y_test = scaler.inverse_transform(y)

# Compute micro-average ROC curve and ROC area
fpr, tpr, _ = roc_curve(np.array(y_val).flatten(), y_pred.flatten())
roc_auc = auc(fpr, tpr)

# Plot micro-average ROC curve
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label='ROC curve (area = {0:0.2f})'
        ''.format(roc_auc), color='blue', linewidth=1)

plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
# plt.savefig(savefig)
plt.show()

### PR curve

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc, f1_score
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(np.array(y_val).flatten(), y_pred.flatten())
auc = auc(recall, precision)

# plot the precision-recall curves
no_skill = (np.array(y_val).flatten()).sum() / len(np.array(y_val).flatten())
plt.plot([0, 1], [no_skill,no_skill], linestyle='--', label='positive class = %.3f' % (no_skill))
plt.plot(recall, precision, label='PR_curve, AUC = %.3f' % (auc))
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR Curve')
# show the legend
plt.legend()
# show the plot
plt.show()

### plot predicted against true

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Reshape data to grid
y_pred = np.reshape(y_pred, (y_pred.shape[0], tensor.shape[1], tensor.shape[2]))
true = np.reshape(y_val, (y_val.shape[0], tensor.shape[1], tensor.shape[2]))

# Choose timesteps to plot
timestep = 8


# Extract the data for the chosen timesteps from the tensor
data1 = y_pred[timestep, :, :]
data2 = true[timestep, :, :]

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4))

# Plot the data in each subplot
sns.heatmap(data1, cmap='coolwarm', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax1)
sns.heatmap(data2, cmap='coolwarm', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax2)

# Set the plot titles and axis labels
ax1.set_title(f'Predicted M{cutoff} at timestep {timestep}')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')

# lon_centers = [(interval.left + interval.right) / 2 for interval in grouped.index.levels[0]]
# lat_centers = [(interval.left + interval.right) / 2 for interval in grouped.index.levels[1]]
ax1.set_xticklabels(grouped.index.levels[0], rotation=90)
ax1.set_yticklabels(grouped.index.levels[1].sort_values(ascending=False), rotation=0)

ax2.set_title(f'True M{cutoff} at timestep {timestep}')
ax2.set_xticklabels(grouped.index.levels[0], rotation=90)
ax2.set_yticklabels([])
ax2.set_xlabel('Longitude')
# ax2.set_ylabel(['Latitude'])

# Show the plot
plt.savefig("graphs/LSTM_72M_5x5_1M_M6_t8", bbox_inches="tight")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Reshape data to grid
y_pred = np.reshape(y_pred, (y_pred.shape[0], tensor.shape[1], tensor.shape[2]))
true = np.reshape(y_val, (y_val.shape[0], tensor.shape[1], tensor.shape[2]))

# Choose timesteps to plot
timestep = 3


# Extract the data for the chosen timesteps from the tensor
data1 = y_pred[timestep, :, :]
data2 = true[timestep, :, :]

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4))

# Plot the data in each subplot
sns.heatmap(data1, cmap='coolwarm', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax1)
sns.heatmap(data2, cmap='coolwarm', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax2)

# Set the plot titles and axis labels
ax1.set_title(f'Predicted M{cutoff} at timestep {timestep}')
ax2.set_title(f'True M{cutoff} at timestep {timestep}')
ax1.set_xlabel('Longitude')
ax2.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')

# Set the tick labels for the x and y axes of ax1
ax1.set_xticks(np.arange(0.5, data1.shape[1], 1))
ax1.set_xticklabels(grouped.index.levels[0][::1], rotation=90)
ax1.set_yticks(np.arange(0.5, data1.shape[0], 1))
ax1.set_yticklabels(grouped.index.levels[1][::-1].sort_values(ascending=False), rotation=0)

# Set the tick labels for the x axis of ax2
ax2.set_xticks(np.arange(0.5, data2.shape[1], 1))
ax2.set_xticklabels(grouped.index.levels[0][::1], rotation=90)
ax2.set_yticklabels([])



# Show the plot
plt.savefig("graphs/LSTM_72M_1M_5x5_M6_t4", bbox_inches="tight")
plt.show()